<a href="https://colab.research.google.com/github/pipeberrio/Analitica/blob/main/Tarea3_Anal%C3%ADtica_de_Negocios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tarea #3**


Raul Felipe Berrio Gomez CC *1035416370*

Se identifican los valores faltantes para las columnas a trabajar

In [ ]:
import pandas as pd

!gdown 110nTRpCHddZGW-dXA2d6M645IjFPCaMG

df = pd.read_excel('datos_pqr.xlsx')
df = df.rename(columns={
         'Radicado':'radicado',
         'No. codificación de registro de inscripción ':'registro de inscripción',
         'Mes/Año inscripción':'fecha',
         'Autoridad Ambiental Competente':'jurisdiccion',
         'Nombre o razon social':'razon social',
         'Número de identificación o NIT':'NIT',
         'Representante legal':'representante',
         'Dirección, física de la planta de transformación':'direccion',
         'Teléfono':'telefono',
         'Correo electrónico':'correo',
         'Capacidad instalada de transformación (ton/año)':'capacidad',
         'Tipo de material a transformar':'material',
         'Tipo de transformación':'transformacion',
         'No. del acto adminitrativo  por la cual se otorga permisos, concesiones y autorizaciónes del transformador  (si aplica)': 'No_acto',
         'Fecha del acto administrativo (si aplica)':'fecha_acto',
         'Descripción de la actividad autorizada mediante permisos, concesiones y autorizaciónes del transformador  (si aplica)':'descripcion_act',
         'Observaciones': 'observaciones',
         'Responsable reporte':'responsable_reporte'
})
df


Renombrar las columnas del DF

In [ ]:


# Renombrar las columnas
df = df.rename(columns={
    'Radicado':'radicado',
    'No. codificación de registro de inscripción ':'registro_inscripcion',
    'Mes/Año inscripción':'fecha',
    'Autoridad Ambiental Competente':'jurisdiccion',
    'Nombre o razon social':'razon social',
    'Número de identificación o NIT':'NIT',
    'Representante legal':'representante',
    'Dirección, física de la planta de transformación':'direccion',
    'Teléfono':'telefono',
    'Correo electrónico':'correo',
    'Capacidad instalada de transformación (ton/año)':'capacidad',
    'Tipo de material a transformar':'material',
    'Tipo de transformación':'transformacion'
})

# Verificar cuántos valores faltantes hay en cada una de las columnas especificadas
columnas_interes = ['razon social', 'NIT', 'representante', 'direccion', 'telefono', 'correo', 'capacidad', 'material', 'transformacion']
valores_faltantes = df[columnas_interes].isnull().sum()

print("Valores faltantes por columna:")
print(valores_faltantes)


Una vez identificados se eliminaran las filas donde hay valores faltantes

In [ ]:
# Eliminar las filas que tengan valores faltantes en las columnas de interés
df_limpio = df.dropna(subset=columnas_interes)

# Verificar que no haya valores faltantes en las columnas de interés
valores_faltantes_limpio = df_limpio[columnas_interes].isnull().sum()

print("Valores faltantes después de la limpieza:")
print(valores_faltantes_limpio)

# Mostrar el número de filas antes y después de la limpieza
print(f"Número de filas antes de la limpieza: {len(df)}")
print(f"Número de filas después de la limpieza: {len(df_limpio)}")


Ahora se prosigue con el tratamiento de datos duplicados

In [ ]:
# Identificar filas duplicadas basadas en las columnas de interés
duplicados = df_limpio.duplicated(subset=columnas_interes)

# Mostrar cuántos duplicados hay
print(f"Número de filas duplicadas: {duplicados.sum()}")

Se eliminan las filas con datos duplicados

In [ ]:
# Eliminar duplicados
df_sin_duplicados = df_limpio.drop_duplicates(subset=columnas_interes)

# Mostrar el número de filas antes y después de eliminar duplicados
print(f"Número de filas antes de eliminar duplicados: {len(df_limpio)}")
print(f"Número de filas después de eliminar duplicados: {len(df_sin_duplicados)}")

In [ ]:
# Convertir la columna 'capacidad' a numérica, eliminando cualquier valor no convertible
df_sin_duplicados['capacidad'] = pd.to_numeric(df_sin_duplicados['capacidad'], errors='coerce')

# Calcular el primer cuartil (Q1) y el tercer cuartil (Q3)
Q1 = df_sin_duplicados['capacidad'].quantile(0.25)
Q3 = df_sin_duplicados['capacidad'].quantile(0.75)
IQR = Q3 - Q1

# Definir los límites para identificar outliers
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Filtrar los datos para excluir outliers
df_sin_outliers = df_sin_duplicados[
    (df_sin_duplicados['capacidad'] >= limite_inferior) &
    (df_sin_duplicados['capacidad'] <= limite_superior)
]

# Mostrar cuántas filas tenían outliers
print(f"Número de filas antes de eliminar outliers: {len(df_sin_duplicados)}")
print(f"Número de filas después de eliminar outliers: {len(df_sin_outliers)}")
print(f"Outliers eliminados: {len(df_sin_duplicados) - len(df_sin_outliers)}")


pd.to_numeric(..., errors='coerce'): Convertimos la columna capacidad a un tipo numérico, reemplazando cualquier valor no numérico con NaN.


quantile(0.25) y quantile(0.75): Calculan el primer cuartil (Q1) y el tercer cuartil (Q3), necesarios para identificar el IQR.
Límites de outliers: Se establecen como 1.5 veces el IQR por debajo de Q1 y por encima de Q3.


Filtrado: Se eliminan las filas cuyo valor de capacidad está fuera de estos límites.

Normalización de columnas

Normalización de la columna direccion:
El objetivo es garantizar que las direcciones sigan un formato coherente, como asegurarse de que no haya espacios en blanco adicionales, caracteres innecesarios o que sigan un formato claro.

In [ ]:
import re

# Función para estandarizar las direcciones
def estandarizar_direccion(direccion):
    if pd.isnull(direccion):
        return direccion  # Si la dirección es nula, no hacer nada
    direccion = direccion.strip()  # Eliminar espacios al inicio y al final
    direccion = direccion.upper()  # Convertir a mayúsculas
    direccion = re.sub(r'\bCRA\b', 'CARRERA', direccion)  # Reemplazar "CRA" por "CARRERA"
    direccion = re.sub(r'#', '', direccion)  # Eliminar el símbolo "#"
    direccion = re.sub(r'-', '', direccion)  # Eliminar el símbolo "-"
    direccion = direccion.replace('.', '')  # Eliminar el carácter "."
    direccion = re.sub(r',', '', direccion)  # Eliminar el símbolo ","
    direccion = re.sub(r'\bNO\b', '', direccion)  # Eliminar "NO"
    direccion = re.sub(r'\bN°\b', '', direccion)  # Eliminar "N°"
    direccion = re.sub(r'\bNº\b', '', direccion)  # Eliminar "Nº"
    direccion = re.sub(r'\bª\b', '', direccion)  # Eliminar "ª"
    direccion = direccion.replace('CARRRERA', 'CARRERA')
    direccion = direccion.replace('CALLE', 'CALLE')
    direccion = direccion.replace('KM', 'KILOMETRO')
    direccion = direccion.replace('NRO', 'NÚMERO')
    direccion = direccion.replace('NO', 'NÚMERO')
    direccion = re.sub(r'\s+', ' ', direccion)  # Eliminar múltiples espacios y dejar solo uno
    return direccion

 # Reemplazar múltiples espacios por un solo espacio
    direccion = re.sub(r'\s+', ' ', direccion)

    # Eliminar espacios al inicio y al final
    direccion = direccion.strip()

    return direccion

# Supongamos que la columna de direcciones se llama 'direccion'
df['direccion'] = df['direccion'].apply(estandarizar_direccion)

# Guardar el DataFrame limpio
df.to_csv('direcciones_limpias.csv', index=False)
# Aplicar la función de normalización a la columna 'direccion'
df_sin_outliers['direccion'] = df_sin_outliers['direccion'].apply(estandarizar_direccion)

# Mostrar algunas direcciones normalizadas
df_sin_outliers['direccion'].head()
df_sin_outliers


Normalización de la columna correo:

Para los correos electrónicos, nos aseguraremos de que no tengan espacios innecesarios y que el formato sea consistente.



In [ ]:
import pandas as pd
import re
import numpy as np  # Importar numpy para usar np.nan

# Función para estandarizar los correos electrónicos
def estandarizar_correo(correo):
    if pd.isnull(correo):
        return correo  # Si el correo es nulo, no hacer nada
    correo = correo.strip()  # Eliminar espacios al inicio y al final
    correo = correo.lower()  # Convertir a minúsculas
    # Verificar el formato básico de un correo electrónico
    if re.match(r'^[\w\.-]+@[\w\.-]+\.\w+$', correo):
        return correo
    else:
        return None  # Retornar None si el correo no es válido

# Aplicar la función de normalización a la columna 'correo' usando .loc
df_sin_outliers.loc[:, 'correo'] = df_sin_outliers['correo'].apply(estandarizar_correo)

# Reemplazar None por NaN
df_sin_outliers['correo'].replace({None: np.nan}, inplace=True)

# Mostrar algunos correos normalizados
print(df_sin_outliers['correo'].head())

# Verificar el número de filas y columnas después de la normalización
print(f"Filas y columnas después de la normalización del correo: {df_sin_outliers.shape}")





# Mostrar valores únicos de las columnas 'material' y 'transformacion'
print("Valores únicos en 'material':")
print(df_sin_outliers['material'].unique())

print("\nValores únicos en 'transformacion':")
print(df_sin_outliers['transformacion'].unique())

# Función para normalizar materiales
def estandarizar_material(material):
    if pd.isnull(material):
        return material  # Si el material es nulo, no hacer nada
    material = material.strip()  # Eliminar espacios al inicio y al final
    material = material.lower()  # Convertir a minúsculas
    # Reemplazar nombres similares o errores tipográficos
    # Ejemplo de normalización, ajusta según tus necesidades
    material = material.replace('plastic', 'plástico')
    material = material.replace('metals', 'metales')
    material = material.replace('glass', 'vidrio')
    # Añadir más reglas según sea necesario
    return material

# Función para normalizar procesos de transformación
def estandarizar_transformacion(transformacion):
    if pd.isnull(transformacion):
        return transformacion  # Si la transformación es nula, no hacer nada
    transformacion = transformacion.strip()  # Eliminar espacios al inicio y al final
    transformacion = transformacion.lower()  # Convertir a minúsculas
    # Ejemplo de normalización, ajusta según tus necesidades
    transformacion = transformacion.replace('recycle', 'reciclaje')
    transformacion = transformacion.replace('reuse', 'reutilización')
    # Añadir más reglas según sea necesario
    return transformacion

# Aplicar las funciones de normalización a las columnas correspondientes
df_sin_outliers['material'] = df_sin_outliers['material'].apply(estandarizar_material)
df_sin_outliers['transformacion'] = df_sin_outliers['transformacion'].apply(estandarizar_transformacion)

# Mostrar algunos ejemplos normalizados
print("\nMateriales normalizados:")
print(df_sin_outliers['material'].head())

print("\nTransformaciones normalizadas:")
print(df_sin_outliers['transformacion'].head())

df_sin_outliers


Para ajustar la información en las columnas de material y transformacion, primero se debe decidir cómo normalizar y estandarizar los datos.

Estos son los pasos que se siguieron:

Listar los Valores Únicos: Primero, se revisan los valores únicos en las columnas para ver cómo están escritos y detectar inconsistencias.
Definir Reglas de Normalización: Decidir cómo se vera cada material y proceso de transformación (por ejemplo, convertir todo a minúsculas, corregir errores tipográficos, estandarizar abreviaturas, etc.).
Aplicar la Normalización: Se utiliza una función similar a la que se uso para las direcciones y correos.

In [ ]:
import pandas as pd

# Mostrar valores únicos de las columnas 'material' y 'transformacion'
print("Valores únicos en 'material':")
print(df_sin_outliers['material'].unique())

print("\nValores únicos en 'transformacion':")
print(df_sin_outliers['transformacion'].unique())

# Función para normalizar materiales
def estandarizar_material(material):
    if pd.isnull(material):
        return material  # Si el material es nulo, no hacer nada
    material = material.strip()  # Eliminar espacios al inicio y al final
    material = material.lower()  # Convertir a minúsculas
    # Reemplazar nombres similares o errores tipográficos
    # Ejemplo de normalización, ajusta según tus necesidades
    material = material.replace('plastic', 'plástico')
    material = material.replace('metals', 'metales')
    material = material.replace('glass', 'vidrio')
    # Añadir más reglas según sea necesario
    return material

# Función para normalizar procesos de transformación
def estandarizar_transformacion(transformacion):
    if pd.isnull(transformacion):
        return transformacion  # Si la transformación es nula, no hacer nada
    transformacion = transformacion.strip()  # Eliminar espacios al inicio y al final
    transformacion = transformacion.lower()  # Convertir a minúsculas
    # Ejemplo de normalización, ajusta según tus necesidades
    transformacion = transformacion.replace('recycle', 'reciclaje')
    transformacion = transformacion.replace('reuse', 'reutilización')
    # Añadir más reglas según sea necesario
    return transformacion

# Aplicar las funciones de normalización a las columnas correspondientes
df_sin_outliers['material'] = df_sin_outliers['material'].apply(estandarizar_material)
df_sin_outliers['transformacion'] = df_sin_outliers['transformacion'].apply(estandarizar_transformacion)

# Mostrar algunos ejemplos normalizados
print("\nMateriales normalizados:")
print(df_sin_outliers['material'].head())

print("\nTransformaciones normalizadas:")
print(df_sin_outliers['transformacion'].head())

df_sin_outliers




Se elimina el caracter "." de la Columna NIT para hacer mas facil el tratamiento de los datos

In [ ]:
# Eliminar el carácter "." de la columna NIT
df_sin_outliers['NIT'] = df_sin_outliers['NIT'].str.replace('.', '', regex=False)

# Verificar las primeras filas para confirmar los cambios
df_sin_outliers['NIT'].head()

df_sin_outliers

Se valida la cantidad de filas totales despues del preprocesamiento de los datos

In [ ]:
# Mostrar la cantidad total de filas del DataFrame final
total_filas = df.shape[0]
print(f"Cantidad total de filas en el DataFrame final: {total_filas}")

Se genera el archivo en excel y su respectiva descarga

In [ ]:
# Guardar el DataFrame como archivo Excel
archivo_salida = 'datos_preprocesados.xlsx'
df_sin_outliers.to_excel(archivo_salida, index=False)

# Descargar el archivo en Google Colab
from google.colab import files
files.download(archivo_salida)